In [8]:
import wikipedia
import wikipedia.exceptions as wiki_exceptions
import re
import spacy
from nltk.tokenize import sent_tokenize
import sys

In [9]:
nlp=spacy.load("en_core_web_sm")

In [24]:
class QASystem:
    def __init__(self, logfile): 
        self.logfile = logfile
    def run(self):
        """run qa system interactively"""
    print("***this is qa system by team 5")
    while True:
        try:
            question = input ("*?>").strip()
            if question.lower() == "exit":
                print("Thank you, goodbye")
                break
            self.answer_question(question)
        except Exception as e:
            print(f"An error occurred: {e}")
            continue
def answer_question(self, question):
    """determine question type"""
    doc = nlp(question)

    #id aquestion and clean it
    question_type = self.identify_question_type(question)
    if question_type is None:
        print("i am sorry don't know answer")
        self.log_question(question, "n/a")
        return 

    ##use extract_context method to refine query
    refined_query = self.extract_context(question)
    if not refined_query:
        refined_query = self.extract_dynamic_entity(doc, question_type)
    if refined_query: #look for wikipedia search query
        print(f"attempting ot search wikipedia for : {refined_query}")
        self.search_wikipedia(refined_query, question_type, question)
    else: #if no refined query
        print("Sorry")
def extract_context(self, question): # regex patterns to get common q's
    """extract relevant subject using regex"""
    patterns = [
        r'where (Is|Was|is|was|are|did) (.+)',
        r'Who (Is|Was|are|) (.+)',
        r'what (Is|Was) ( .* ) Age',
        r'What (Is|Was) (.+)',
        r'When (Is|Was) ( .. ) Born',
        r'When (Is|Was) (.+) Birthday',
        r'Where (.+)',
        r'who (.+)',
        r'What (.+)',
        r'when (.+) Born',
        r'When (.+) Birthday',
        r'When did (.+)'
    ]

    for pattern in patterns:
        match = re.match(patttern, question, re.IGNORECASE)
        if match:
            return match.group(2).strip()
    return None
def identify_question_type(self, question): #classify the question
    """id whetehr who, what when where"""
    question_lower = question.lower()
    if question_lower.startswith("who"):
        return "Who"
    elif question_lower.startswith("what"):
        return "What"
    elif question_lower.startswith("when"):
        return "When"
    elif question_lower.startswith("where"):
        return "Where"
    return None

def extract_dynamic_entity(self, doc, question_type):
    """dynamically extract rlevant parts using NER"""
    entities = [ent.text for ent in doc.ents if ent.label_ in {"PERSON", "ORG","GPE", "DATE"}]
    if entities:
        return ' '.join(entities)
    #if new doesn find enties, clean based on type
    question - re.sub(r'[^a-zA-A0-9\s]',  '',doc.text).strip() #remove special characters
    return question.lower()

def search_wikipedia(self, query, question_type, question):
    """search wikipedia and summarize contect based on query type"""
    try:
        search_results = wikipedia.search(query) # searching wikipedia
        if search_results:
            #retireve and print summary of first result
            summary = wikipedia.summary(search_results[0], sentences = 5)
            meaningful_summary = self.summarize_text(summary, question_type, query)#customize summary based
            if meaningful_summary:
                print(f"=> {meaningful_summary}")
                self.log_question(question, meaningful_summary)
            else:
                print("i am sorry, don't know answer")
                self.log_question(question, "N/A")
        else:
            print("I am sorry don't know answer.")
            self.log_question(questino, "n/a")
    except wiki_exceptions.DisambiguationError as e: #ambigious answers
        print("question is amigous here are some possible answers")
        for option in e.options[:5]:
            print(f"-{option}")
        self.log_question(question, "ambiguous")

    except wiki_exceptions.PageError:
        print("couldn't find page on htat topic")
        self.log_question(question, "no result")
    except wiki_exceptions.HTTPTimeoutError:
        print("network error try latters")
        self.log_question(question,"timeout")

    def summarize_text(self, text, question_type, query):
        """summarize dynamically based on query type"""
        sentences = sent_tokenize(text)
        
        # Use relevant patterns to search for the correct information based on the question type
        if question_type == "Who":
            for sentence in sentences:
                if query in sentence and ("is" in sentence or "was" in sentence):
                    return sentence
            return sentences[0]
        
        elif question_type == "What":
            return sentences[0] # For "What", return the first sentence as a general definition
        
        elif question_type == "When":
            for sentence in sentences:
                if re.search(r"\b\d(4)\b|\bJanuary|\bMonday|\bcentury\b", sentence):
                    return sentence
            return "Date or time information not found."
        
        elif question_type == "Where":
        # Using NER to find location-related entities
            for sentence in sentences:
                if "GPE" in [ent.label_ for ent in nlp(sentence).ents]:
                    return sentence
            return "Location information mot found."
        
    def log_question(self, question, answer):
        """Log the question and the answer to the log file using UTF-8 encoding."""
        with open(self.logfile, "a', encoding='utf-8") as log:
            log.write(f"Question: {question}\n")
            log.write(f"Answer: {answer}\m\n")

# Main function execution with logging
def main():
# Prompt the user to input the Log file name
    log_filename = input("Enter the name of the log file: ").strip()

    try:
        # Create an inctanre of the 0lSystem and cun it
        qa_system= QAsystem(log_filename)
        qa_system.run()
    except Exception as ex:
        print(ex)
    finally:
        print("logging complete.")

if __name__ == "__main__": #run main function
    main()

***this is qa system by team 5


*?> who is ronald regan


An error occurred: name 'self' is not defined


*?> quit


An error occurred: name 'self' is not defined


KeyboardInterrupt: Interrupted by user